# [Std 04] - calculate Christoffel symbols in spherical coords

In [1]:
%display latex
#Parallelism().set(nproc=10)

import numpy as np
import scipy
from numpy import r_

from sage.manifolds.operators import *

First, let's define a Euclidean space in spherical coordinates, over the variables $r$, $\theta$, and $\phi$, with accompanying limits on the domain.  This is accomplished by identifying the accompanying manifold for the problem statement $\mathcal{M}$ over the reals in $\mathbb{R}^3$; which is a pseudo-Riemannian structure with vector magnitude-conservation (length invariance under linear transformation), and can be considered continuous and continuously differentiable (i.e., _smooth_ in $\mathcal{C}^\infty$) at all topological points on the manifold. 

We further introduce a "Chart", surjectively mapping Cartesian coordinates into the spherical coordinates (azimuthal $\phi$, $\cos\theta = \hat{\mathbf{z}}$).

In [2]:
M = Manifold(3, 'M')
M.set_calculus_method('sympy')
spherical_coords.<r, θ, φ> = M.chart(r'r:(0,+oo) θ:(0,pi):\theta φ:(0,2*pi):\phi')

spherical_coords

Chart (M, (r, θ, φ))

To be complete, let's also check out the coordinate basis frames.  These are in order of its coordinate frame (a tuple of its basis and partials), it's basis frame (in spherical coordinates), which is derived as a Chart from the Cartesian coordinates.  Coordinate frames matter in GR, we will address that later. 

In [3]:
coord_frame = M.frames()[0]
coord_frame

Coordinate frame (M, (∂/∂r,∂/∂θ,∂/∂φ))

* c.ref. https://math.stackexchange.com/q/1985964 and https://mathworld.wolfram.com/SphericalCoordinates.html; order of the parameter definitions matter!

* Given a curvilinear world-line path in spherical coordinates $ds^2 = dr^2 + r\ d \theta^2 + r^2 \sin^2 \theta\ d\phi^2$, we define a metric
    $$ g^{ij} = \begin{pmatrix} 1 & 0 & 0 \\ 0 & r^2 & 0 \\ 0 & 0 & r^2 \sin^2\theta \end{pmatrix}$$

* We also define the Christoffel symbols in summation-notation: $$ \Gamma_{jk}^{i} \equiv \frac{1}{2} \left( g^{-1} \right)^{il} \left[ \frac{\partial g^{kl}}{\partial x^j} + \frac{\partial g^{jl}}{\partial x^k} - \frac{\partial g^{jk}}{\partial x^l} \right] $$ or equivalently $$ \Gamma_{jk}^i \equiv \frac{1}{2} \left( g^{-1} \right)^{il} \left[ \partial_j g^{kl} + \partial_k g^{jl} - \partial_l g^{jk} \right] $$

* **QUESTION :>** in the definitions above, are the coordinate indices supposed to be upper or lower? Is the definition valid for the contravariant, or the covariant, expression of the indices? Differing definitions across sources. 

In [4]:
# For our variable names, we'll use Unicode U+2054 (CHARACTER TIE) to 
#  indicate contravariant (upper) indices, followed by an underscore preceding
#  covariant indices.
g⁀ij = M.metric('g')
g⁀ij[0,0], g⁀ij[1,1], g⁀ij[2,2] = 1, r^2, r^2 * (sin(θ))^2
g⁀ij.display()

g = dr⊗dr + r**2 dθ⊗dθ + r**2*sin(θ)**2 dφ⊗dφ

In [5]:
g⁀ij[:]

[             1              0              0]
[             0           r**2              0]
[             0              0 r**2*sin(θ)**2]

In [6]:
g⁀ij.display()

g = dr⊗dr + r**2 dθ⊗dθ + r**2*sin(θ)**2 dφ⊗dφ

In [7]:
# check that the metric has no curvature, like the Minkowski metric
g⁀ij.riemann().display()

Riem(g) = 0

In [8]:
from sage.tensor.modules.tensor_with_indices import TensorWithIndices

g_inv = g⁀ij.inverse()
g_inv = TensorWithIndices(g_inv, '^ij')
g_inv[:]

[                 1                  0                  0]
[                 0            r**(-2)                  0]
[                 0                  0 1/(r**2*sin(θ)**2)]

We are now able to phrase the Christoffel symbols as contractions on the indices, following the definition.  It turns out that Sage Math incorporates a function to handle this for us; after noting that due to symmetries, some of the terms cancel

In [9]:
g⁀ij.christoffel_symbols_display(coordinate_labels=False)

Gam^0_11 = -r 
Gam^0_22 = -r*sin(θ)**2 
Gam^1_01 = 1/r 
Gam^1_22 = -sin(2*θ)/2 
Gam^2_02 = 1/r 
Gam^2_12 = 1/tan(θ)

It's important to note that we have assumed symmetry of the upper and lower indices of the Christoffel symbols, due to a "existence and uniqueness"-style argument.  The Christoffel symbols themselves arise by solving the corresponding _parallel transport_ problem for the same metric space, after chart mapping.  The Christoffel symbols themselves describe a functional mapping in the parallel transport problem that encode functions of some displacement or position, which is related to derivatives of the metric tensor.  We can see this arise in a differential expression for the parallel transport problem; $$\delta A_\mu = \Gamma^\nu_{\mu \beta} A_\nu dx^\beta$$

It turns out that these symbols (Christoffel, Levi-Civita) describe "connections" between metric spaces that preserve qualities of the encoding of the metric, and are related to the gradient and divergence (respectively) of the metric.  Unfortunately, Sage Math doesn't directly implement derivatives using tensor notation, and instead uses the information encoded in the supplied metric $g^{ij}$ to compute properties of the corresponding connection tensor, from which it computes the Christoffel symbols. So, no exact computation here. 

In [10]:
g⁀ij.connection()

Levi-Civita connection nabla_g associated with the Riemannian metric g on the 3-dimensional differentiable manifold M

In [11]:
g⁀ij.connection().display()

Gam^r_θθ = -r 
Gam^r_φφ = -r*sin(θ)**2 
Gam^θ_rθ = 1/r 
Gam^θ_θr = 1/r 
Gam^θ_φφ = -sin(2*θ)/2 
Gam^φ_rφ = 1/r 
Gam^φ_θφ = 1/tan(θ) 
Gam^φ_φr = 1/r 
Gam^φ_φθ = 1/tan(θ)